![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.9.German_Clinical_Deidentification.ipynb)


If you are using the `johnsnowlabs` library, please use this  [04.9.German_Clinical_Deidentification](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/04.9.German_Clinical_Deidentification.ipynb) notebook.

## Start Spark Session

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1 spark-nlp==$PUBLIC_VERSION


# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp_jsl.pipeline_tracer import PipelineTracer
from sparknlp_jsl.pipeline_output_parser import PipelineOutputParser

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import pandas as pd
pd.set_option('display.max_colwidth', 0)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark


Spark NLP Version : 6.1.3
Spark NLP_JSL Version : 6.1.1


In [ ]:
def get_results(text, pipeline):

    deid_res = pipeline.fullAnnotate(text)

    for res in deid_res:
        sentence = [original_text.result for original_text in res["document"]]
        masked = [masked_text.result for masked_text in res["masked"]]
        obfuscated = [obfuscated_text.result for obfuscated_text in res["obfuscated"]]

    return pd.DataFrame({"Sentence": sentence, "Masked": masked, "Obfuscated":obfuscated})

# German `Deidentification` Pipeline

In [ ]:
from sparknlp.pretrained import PretrainedPipeline
deid_pipe = PretrainedPipeline("clinical_deidentification_docwise_wip", "de", "clinical/models")

clinical_deidentification_docwise_wip download started this may take some time.
Approx size to download 1.2 GB
[OK!]


## Text-1

In [ ]:
sample_text_1 = """
Klinischer Bericht
Patienten-ID: 12345678
Name: Herr Max Mustermann
Geburtsdatum: 12 März 1985
Adresse: Musterstraße 12, 12345 Stuttgart
Stadt: Berlin
Telefonnummer: +49 30 12345678
E-Mail: max.mustermann@example.de
Sozialversicherungsnummer (SVN): 13110587M565
Aufnahmedatum: 20 Oktober 2024
Entlassungsdatum: 25 Oktober 2024

Diagnose:
ICD-10: E11.9 (Typ-2-Diabetes mellitus, nicht näher bezeichnet)
OPS-Code: 5-470.01 (Transurethrale Resektion der Prostata)

Medikamente:
PZN: 12345678 (Metformin 850 mg Tabletten)
PZN: 87654321 (Aspirin 100 mg Tabletten)

Hilfsmittel:
HMV: 17.99.09.1010 (Blutzuckermessgerät)

Kontaktperson: Frau Anna Mustermann
Kontaktadresse: Blumenstraße 5, 12345 Berlin
Kontaktnummer: +49 3098765432

Verlauf:
Der Patient wurde am 20.10.2024 aufgrund erhöhter Blutzuckerwerte aufgenommen.
Bei der Aufnahme wurden folgende Vitalwerte gemessen: Blutdruck 140/90 mmHg, Blutzucker 280 mg/dL. Eine transurethrale Resektion der Prostata (OPS 5-470.01) wurde am 22.10.2024 erfolgreich durchgeführt.
Der Patient zeigte nach der Operation keine Komplikationen und wurde am 25.10.2024 entlassen.
"""

get_results(sample_text_1, deid_pipe)

,Sentence,Masked,Obfuscated
0,"\nKlinischer Bericht\nPatienten-ID: 12345678\nName: Herr Max Mustermann\nGeburtsdatum: 12 März 1985\nAdresse: Musterstraße 12, 12345 Stuttgart\nStadt: Berlin\nTelefonnummer: +49 30 12345678\nE-Mail: max.mustermann@example.de\nSozialversicherungsnummer (SVN): 13110587M565\nAufnahmedatum: 20 Oktober 2024\nEntlassungsdatum: 25 Oktober 2024\n\nDiagnose:\nICD-10: E11.9 (Typ-2-Diabetes mellitus, nicht näher bezeichnet)\nOPS-Code: 5-470.01 (Transurethrale Resektion der Prostata)\n\nMedikamente:\nPZN: 12345678 (Metformin 850 mg Tabletten)\nPZN: 87654321 (Aspirin 100 mg Tabletten)\n\nHilfsmittel:\nHMV: 17.99.09.1010 (Blutzuckermessgerät)\n\nKontaktperson: Frau Anna Mustermann\nKontaktadresse: Blumenstraße 5, 12345 Berlin\nKontaktnummer: +49 3098765432\n\nVerlauf:\nDer Patient wurde am 20.10.2024 aufgrund erhöhter Blutzuckerwerte aufgenommen.\nBei der Aufnahme wurden folgende Vitalwerte gemessen: Blutdruck 140/90 mmHg, Blutzucker 280 mg/dL. Eine transurethrale Resektion der Prostata (OPS 5-470.01) wurde am 22.10.2024 erfolgreich durchgeführt.\nDer Patient zeigte nach der Operation keine Komplikationen und wurde am 25.10.2024 entlassen.\n","\nKlinischer Bericht\nPatienten-ID: <ID>\nName: Herr <NAME> <NAME>\nGeburtsdatum: <DATE>\nAdresse: Musterstraße 12, <ZIP> <CITY>\nStadt: <STATE>\nTelefonnummer: <PHONE>\nE-Mail: <EMAIL>\nSozialversicherungsnummer (SVN): <SSN>Aufnahmedatum: <DATE>\nEntlassungsdatum: <DATE>\n\nDiagnose:\nICD-10: E11.9 (Typ-2-Diabetes mellitus, nicht näher bezeichnet)\nOPS-Code: 5-470.01 (Transurethrale Resektion der Prostata)\n\nMedikamente:\nPZN: 12345678 (Metformin 850 mg Tabletten)\nPZN: 87654321 (Aspirin 100 mg Tabletten)\n\nHilfsmittel:\nHMV: 17.99.09.1010 (Blutzuckermessgerät)\n\nKontaktperson: Frau <NAME> <NAME>\nKontaktadresse: Blumenstraße 5, <ZIP> <STATE>\nKontaktnummer: <PHONE>\n\nVerlauf:\nDer Patient wurde am <DATE> aufgrund erhöhter Blutzuckerwerte aufgenommen.\nBei der Aufnahme wurden folgende Vitalwerte gemessen: Blutdruck 140/90 mmHg, Blutzucker 280 mg/dL. Eine transurethrale Resektion der Prostata (OPS 5-470.01) wurde am <DATE> erfolgreich durchgeführt.\nDer Patient zeigte nach der Operation keine Komplikationen und wurde am <DATE> entlassen.\n","\nKlinischer Bericht\nPatienten-ID: 85296307\nName: Herr Marcella Constanze\nGeburtsdatum: 11 Mai 1985\nAdresse: Musterstraße 12, 85296 Pößneck\nStadt: Saarland\nTelefonnummer: +94 21 85296307\nE-Mail: huan@google.com\nSozialversicherungsnummer (SVN): 82881670B636Aufnahmedatum: 19 Dezember 2024\nEntlassungsdatum: 24 Dezember 2024\n\nDiagnose:\nICD-10: E11.9 (Typ-2-Diabetes mellitus, nicht näher bezeichnet)\nOPS-Code: 5-470.01 (Transurethrale Resektion der Prostata)\n\nMedikamente:\nPZN: 12345678 (Metformin 850 mg Tabletten)\nPZN: 87654321 (Aspirin 100 mg Tabletten)\n\nHilfsmittel:\nHMV: 17.99.09.1010 (Blutzuckermessgerät)\n\nKontaktperson: Frau Ina Constanze\nKontaktadresse: Blumenstraße 5, 85296 Saarland\nKontaktnummer: +94 2147036925\n\nVerlauf:\nDer Patient wurde am 19.12.2024 aufgrund erhöhter Blutzuckerwerte aufgenommen.\nBei der Aufnahme wurden folgende Vitalwerte gemessen: Blutdruck 140/90 mmHg, Blutzucker 280 mg/dL. Eine transurethrale Resektion der Prostata (OPS 5-470.01) wurde am 21.12.2024 erfolgreich durchgeführt.\nDer Patient zeigte nach der Operation keine Komplikationen und wurde am 24.12.2024 entlassen.\n"


## Text-2

In [ ]:
sample_text_2 = """Sehr geehrter Herr Schmidt, bezüglich Ihrer Anfrage vom 15.03.2024 für Ihre Krankenakte
341123 möchte ich bestätigen, dass Ihre Sozialversicherungsnummer 13110587M565 und
Ihre Versichertennummer T0110053F5D korrekt in unserem System hinterlegt sind. Wie besprochen können Sie Ihre
neue Fahrerlaubnis B072RRE2I55 in der Behörde in München, Bayern, Deutschland abholen. Die Adresse dafür
ist Musterstraße 12, 12345 Stuttgart. Ihr Fahrzeug mit dem Kennzeichen M-AB 1234  ist bereits registriert.
Falls Sie Fragen haben, erreichen Sie uns unter +49 89 12345678 oder schreiben Sie an support@beispiel.de.
Unsere Details finden Sie auch unter https://www.beispiel-behoerde.de. Für Ihre Bankgeschäfte nutzen
Sie weiterhin das Konto DE89 3704 0044 0532 0130 00. Laut unseren Unterlagen
sind Sie 45 Jahre alt. Zertifikat: 12345As. Alle Ihre Daten werden gemäß der DSGVO in unserer Zentrale in der
Robert-Koch-Straße 45, 80335 München verarbeitet. Alternativ können Sie auch unsere Zweigstelle
in Berlin oder Hamburg besuchen. Zusätzliche Informationen finden Sie auf unserem Portal
kontaktieren Sie uns unter kontakt@beispiel-support.de. Fahrzeug-Identifizierungsnummer: 4Y1SL65848Z411439,"""

get_results(sample_text_2, deid_pipe)

,Sentence,Masked,Obfuscated
0,"Sehr geehrter Herr Schmidt, bezüglich Ihrer Anfrage vom 15.03.2024 für Ihre Krankenakte\n341123 möchte ich bestätigen, dass Ihre Sozialversicherungsnummer 13110587M565 und\nIhre Versichertennummer T0110053F5D korrekt in unserem System hinterlegt sind. Wie besprochen können Sie Ihre\nneue Fahrerlaubnis B072RRE2I55 in der Behörde in München, Bayern, Deutschland abholen. Die Adresse dafür\nist Musterstraße 12, 12345 Stuttgart. Ihr Fahrzeug mit dem Kennzeichen M-AB 1234 ist bereits registriert.\nFalls Sie Fragen haben, erreichen Sie uns unter +49 89 12345678 oder schreiben Sie an support@beispiel.de.\nUnsere Details finden Sie auch unter https://www.beispiel-behoerde.de. Für Ihre Bankgeschäfte nutzen\nSie weiterhin das Konto DE89 3704 0044 0532 0130 00. Laut unseren Unterlagen\nsind Sie 45 Jahre alt. Zertifikat: 12345As. Alle Ihre Daten werden gemäß der DSGVO in unserer Zentrale in der\nRobert-Koch-Straße 45, 80335 München verarbeitet. Alternativ können Sie auch unsere Zweigstelle\nin Berlin oder Hamburg besuchen. Zusätzliche Informationen finden Sie auf unserem Portal\nkontaktieren Sie uns unter kontakt@beispiel-support.de. Fahrzeug-Identifizierungsnummer: 4Y1SL65848Z411439,","Sehr geehrter Herr <NAME>, bezüglich Ihrer Anfrage vom <DATE> für Ihre Krankenakte\n<MEDICALRECORD> möchte ich bestätigen, dass Ihre Sozialversicherungsnummer <SSN>und\nIhre Versichertennummer <ID> korrekt in unserem System hinterlegt sind. Wie besprochen können Sie Ihre\nneue Fahrerlaubnis <DLN> in der Behörde in <CITY>, <STATE>, <COUNTRY> abholen. Die Adresse dafür\nist Musterstraße 12, <ZIP> <CITY>. Ihr Fahrzeug mit dem Kennzeichen <PLATE> ist bereits registriert.\nFalls Sie Fragen haben, erreichen Sie uns unter <PHONE> oder schreiben Sie an <EMAIL>.\nUnsere Details finden Sie auch unter <URL>. Für Ihre Bankgeschäfte nutzen\nSie weiterhin das Konto <ACCOUNT>. Laut unseren Unterlagen\nsind Sie <AGE> Jahre alt. Zertifikat: <LICENSE>. Alle Ihre Daten werden gemäß der DSGVO in unserer Zentrale in der\n<STREET>, <ZIP> <CITY> verarbeitet. Alternativ können Sie auch unsere <LOCATION>\nin <STATE> oder <STATE> besuchen. Zusätzliche Informationen finden Sie auf unserem Portal\nkontaktieren Sie uns unter <EMAIL>. Fahrzeug-Identifizierungsnummer: <VIN>,","Sehr geehrter Herr Raymond, bezüglich Ihrer Anfrage vom 14.05.2024 für Ihre Krankenakte\n298852 möchte ich bestätigen, dass Ihre Sozialversicherungsnummer 82881670B636und\nIhre Versichertennummer G1881162W6C korrekt in unserem System hinterlegt sind. Wie besprochen können Sie Ihre\nneue Fahrerlaubnis I105MMZ5N66 in der Behörde in Rochlitz, Rheinland-pfalz, Deutschland abholen. Die Adresse dafür\nist Musterstraße 12, 85296 Pößneck. Ihr Fahrzeug mit dem Kennzeichen B-LI 8529 ist bereits registriert.\nFalls Sie Fragen haben, erreichen Sie uns unter +94 74 85296307 oder schreiben Sie an aeron@google.com.\nUnsere Details finden Sie auch unter politicalmakeover.com.ee. Für Ihre Bankgeschäfte nutzen\nSie weiterhin das Konto CZ74 2019 1199 1625 1821 11. Laut unseren Unterlagen\nsind Sie 59 Jahre alt. Zertifikat: 85296Lj. Alle Ihre Daten werden gemäß der DSGVO in unserer Zentrale in der\nMandy-peukert-weg 3/4, 71226 Rochlitz verarbeitet. Alternativ können Sie auch unsere Wenke-fliegner-straße 44\nin Saarland oder Bremen besuchen. Zusätzliche Informationen finden Sie auf unserem Portal\nkontaktieren Sie uns unter brooks@hotmail.com. Fahrzeug-Identifizierungsnummer: 9R8JE36797O988924,"


## Text-3

In [ ]:
sample_text_3 = """P98765432
15 Oktober 2024
Musterstraße 5, 80331 München
lisa.meier@example.de

Frau Lisa Meier wurde am  ins Krankenhaus in München aufgenommen.  Ihre Sozialversicherungsnummer lautet 26110587K565.
Die Diagnose lautete essentielle Hypertonie (I10) und fortgeschrittene Arthrose im rechten Knie. Am 17 Oktober 2024 erhielt sie erfolgreich eine Kniegelenk-Endoprothese (OPS-Code 5-775.00).
Ihr behandelnder Arzt verschrieb Ramipril 5 mg (PZN: 76543210) und Ibuprofen 400 mg (PZN: 09876543) zur Schmerzbehandlung. Zudem erhielt sie eine Knieorthese (HMV: 23.40.04.2001) zur postoperativen Stabilisierung.
Ihre Kontaktperson, Herr Thomas Meier, wohnhaft in Gartenweg 12, 80333 München, ist unter der Nummer +49 89 123456 erreichbar. Sie ist per E-Mail unter lisa.meier@example.de erreichbar.
Frau Meier wurde am 20 Okt 2024 in gutem Zustand entlassen."""

get_results(sample_text_3, deid_pipe)

,Sentence,Masked,Obfuscated
0,"P98765432\n15 Oktober 2024\nMusterstraße 5, 80331 München\nlisa.meier@example.de\n\nFrau Lisa Meier wurde am ins Krankenhaus in München aufgenommen. Ihre Sozialversicherungsnummer lautet 26110587K565.\nDie Diagnose lautete essentielle Hypertonie (I10) und fortgeschrittene Arthrose im rechten Knie. Am 17 Oktober 2024 erhielt sie erfolgreich eine Kniegelenk-Endoprothese (OPS-Code 5-775.00).\nIhr behandelnder Arzt verschrieb Ramipril 5 mg (PZN: 76543210) und Ibuprofen 400 mg (PZN: 09876543) zur Schmerzbehandlung. Zudem erhielt sie eine Knieorthese (HMV: 23.40.04.2001) zur postoperativen Stabilisierung.\nIhre Kontaktperson, Herr Thomas Meier, wohnhaft in Gartenweg 12, 80333 München, ist unter der Nummer +49 89 123456 erreichbar. Sie ist per E-Mail unter lisa.meier@example.de erreichbar.\nFrau Meier wurde am 20 Okt 2024 in gutem Zustand entlassen.","<ID>\n<DATE>\nMusterstraße 5, <ZIP> <CITY>\n<EMAIL>\n\nFrau <NAME> <NAME> wurde am ins Krankenhaus in <CITY> aufgenommen. Ihre Sozialversicherungsnummer lautet <SSN>\nDie Diagnose lautete essentielle Hypertonie (I10) und fortgeschrittene Arthrose im rechten Knie. Am <DATE> erhielt sie erfolgreich eine Kniegelenk-Endoprothese (OPS-Code 5-775.00).\nIhr behandelnder <PROFESSION> verschrieb Ramipril 5 mg (PZN: 76543210) und Ibuprofen 400 mg (PZN: 09876543) zur Schmerzbehandlung. Zudem erhielt sie eine Knieorthese (HMV: 23.40.04.2001) zur postoperativen Stabilisierung.\nIhre Kontaktperson, Herr <NAME> <NAME>, wohnhaft in <STREET>, <ZIP> <CITY>, ist unter der Nummer <PHONE> erreichbar. Sie ist per E-Mail unter <EMAIL> erreichbar.\nFrau <NAME> wurde am <DATE> in gutem Zustand entlassen.","S47036925\n14 Dezember 2024\nMusterstraße 5, 71228 Rochlitz\nadayre@google.com\n\nFrau Michaele Alina wurde am ins Krankenhaus in Rochlitz aufgenommen. Ihre Sozialversicherungsnummer lautet 53881670H636\nDie Diagnose lautete essentielle Hypertonie (I10) und fortgeschrittene Arthrose im rechten Knie. Am 16 Dezember 2024 erhielt sie erfolgreich eine Kniegelenk-Endoprothese (OPS-Code 5-775.00).\nIhr behandelnder Technischer Support-mitarbeiter verschrieb Ramipril 5 mg (PZN: 76543210) und Ibuprofen 400 mg (PZN: 09876543) zur Schmerzbehandlung. Zudem erhielt sie eine Knieorthese (HMV: 23.40.04.2001) zur postoperativen Stabilisierung.\nIhre Kontaktperson, Herr Erik Alina, wohnhaft in Eimerring 477, 71222 Rochlitz, ist unter der Nummer +94 74 852963 erreichbar. Sie ist per E-Mail unter adayre@google.com erreichbar.\nFrau Alina wurde am 18 Okt 2024 in gutem Zustand entlassen."


## Text-4

In [ ]:
sample_text_4 = """Zusammenfassung : Michael Berger wird am Morgen des 12 Dezember 2018 ins St.Elisabeth Krankenhaus eingeliefert.
Herr Michael Berger ist 76 Jahre alt und hat zu viel Wasser in den Beinen.

Persönliche Daten :
ID-Nummer: T0110053F
Platte A-BC124
Kontonummer: DE89370400440532013000
SSN : 13110587M565
Lizenznummer: B072RRE2I55
Adresse : St.Johann-Straße 13 19300"""

get_results(sample_text_4, deid_pipe)

,Sentence,Masked,Obfuscated
0,Zusammenfassung : Michael Berger wird am Morgen des 12 Dezember 2018 ins St.Elisabeth Krankenhaus eingeliefert.\nHerr Michael Berger ist 76 Jahre alt und hat zu viel Wasser in den Beinen.\n\nPersönliche Daten :\nID-Nummer: T0110053F\nPlatte A-BC124\nKontonummer: DE89370400440532013000\nSSN : 13110587M565\nLizenznummer: B072RRE2I55\nAdresse : St.Johann-Straße 13 19300,Zusammenfassung : <PATIENT> wird am Morgen des <DATE> ins <HOSPITAL> eingeliefert.\nHerr <NAME> <NAME> ist <AGE> Jahre alt und hat zu viel Wasser in den Beinen.\n\nPersönliche Daten :\nID-Nummer: <ID>\nPlatte <PLATE>\nKontonummer: <ACCOUNT>\nSSN : <SSN>Lizenznummer: <DLN>\nAdresse : <STREET> <ZIP>,Zusammenfassung : Zdravko Sommer wird am Morgen des 10 Februar 2019 ins Klinikum Osnabrück eingeliefert.\nHerr Zdravko Sommer ist 71 Jahre alt und hat zu viel Wasser in den Beinen.\n\nPersönliche Daten :\nID-Nummer: G1881162W\nPlatte L-IF859\nKontonummer: CZ74201911991625182111\nSSN : 82881670B636Lizenznummer: I105MMZ5N66\nAdresse : Scheuermannstr. 33 84211


## Text-5

In [ ]:
sample_text_5 = """Um Ihre Identität zu überprüfen, benötigen wir Ihre Kontonummer: DE44 5001 0517 5407 0000 00. Bitte geben Sie auch Ihre Sozialversicherungsnummer (SSN) im Rahmen des Antragsprozesses an: 89110478M657. Bei Fragen können Sie sich gerne an Dr. Anna Schmidt im Gesundheitszentrum Frankfurt wenden, das sich in der 60313 Frankfurt am Main, Hessen, Deutschland befindet. Sie können das Büro auch unter +49-89-12345678 erreichen oder die zweite Filiale in 12345 Hamburg besuchen. Die 28-jährige Frau, die wir zuvor besprochen haben, hat eine Vorgeschichte von Schwangerschaftsdiabetes, der vor acht Jahren diagnostiziert wurde."""

get_results(sample_text_5, deid_pipe)

,Sentence,Masked,Obfuscated
0,"Um Ihre Identität zu überprüfen, benötigen wir Ihre Kontonummer: DE44 5001 0517 5407 0000 00. Bitte geben Sie auch Ihre Sozialversicherungsnummer (SSN) im Rahmen des Antragsprozesses an: 89110478M657. Bei Fragen können Sie sich gerne an Dr. Anna Schmidt im Gesundheitszentrum Frankfurt wenden, das sich in der 60313 Frankfurt am Main, Hessen, Deutschland befindet. Sie können das Büro auch unter +49-89-12345678 erreichen oder die zweite Filiale in 12345 Hamburg besuchen. Die 28-jährige Frau, die wir zuvor besprochen haben, hat eine Vorgeschichte von Schwangerschaftsdiabetes, der vor acht Jahren diagnostiziert wurde.","Um Ihre Identität zu überprüfen, benötigen wir Ihre Kontonummer: <ACCOUNT>. Bitte geben Sie auch Ihre Sozialversicherungsnummer (SSN) im Rahmen des Antragsprozesses an: <SSN> Bei Fragen können Sie sich gerne an Dr. <NAME> <NAME> im Gesundheitszentrum <CITY> wenden, das sich in der <ZIP> <CITY>, <STATE>, <COUNTRY> befindet. Sie können das Büro auch unter <PHONE> erreichen oder die zweite Filiale in <ZIP> <STATE> besuchen. Die <AGE>-jährige Frau, die wir zuvor besprochen haben, hat eine Vorgeschichte von Schwangerschaftsdiabetes, der vor acht Jahren diagnostiziert wurde.","Um Ihre Identität zu überprüfen, benötigen wir Ihre Kontonummer: CZ99 6118 1680 6910 1111 11. Bitte geben Sie auch Ihre Sozialversicherungsnummer (SSN) im Rahmen des Antragsprozesses an: 74881907B360 Bei Fragen können Sie sich gerne an Dr. Ina Raymond im Gesundheitszentrum Wetzlar wenden, das sich in der 31282 Tuttlingen, Nordrhein-westfalen, Deutschland befindet. Sie können das Büro auch unter +94-74-85296307 erreichen oder die zweite Filiale in 85296 Bremen besuchen. Die 21-jährige Frau, die wir zuvor besprochen haben, hat eine Vorgeschichte von Schwangerschaftsdiabetes, der vor acht Jahren diagnostiziert wurde."


## Broken Text 1

In [ ]:
broken_text_1 = """Herr Joha.nnes Be cker (1D: T0110053F5D), 74 Jahre alt, wohnhaft in der Ber!iner Str aße, 50667 Stuttgart, wurd.e am Sep tember 2024 in die K1inik in K öln aufgenom men. Er kLagte über a nhaltende Ma.genbeschwer den und wurde zur Untersuchung eingew1esen. Wäh rend seines Aufenthalts wurde e!ne endosk opische Untersu chung (OPS: 8-980.2) durchgeführt, die keinen pathologischen Be fund erg ab. Zur Behand lun g seines Typ-2-Diabetes (Ell.9) wurde ihm Metformin erordn et. Seine Ehefr au, Frau An na Becke r, wohnhaft in der Rheinstraße 12, ist a!s Notfallkontak t unter der Nummer +49 2211234 errei chbar. Herr Becker wu rde nach erfolgrei cher Stabilisierung seines Gesund heitszustands am 15.ç10.2024 entlas sen."""

get_results(broken_text_1, deid_pipe)

,Sentence,Masked,Obfuscated
0,"Herr Joha.nnes Be cker (1D: T0110053F5D), 74 Jahre alt, wohnhaft in der Ber!iner Str aße, 50667 Stuttgart, wurd.e am Sep tember 2024 in die K1inik in K öln aufgenom men. Er kLagte über a nhaltende Ma.genbeschwer den und wurde zur Untersuchung eingew1esen. Wäh rend seines Aufenthalts wurde e!ne endosk opische Untersu chung (OPS: 8-980.2) durchgeführt, die keinen pathologischen Be fund erg ab. Zur Behand lun g seines Typ-2-Diabetes (Ell.9) wurde ihm Metformin erordn et. Seine Ehefr au, Frau An na Becke r, wohnhaft in der Rheinstraße 12, ist a!s Notfallkontak t unter der Nummer +49 2211234 errei chbar. Herr Becker wu rde nach erfolgrei cher Stabilisierung seines Gesund heitszustands am 15.ç10.2024 entlas sen.","Herr <NAME> <NAME> <PATIENT> (1D: <ID>), <AGE> Jahre alt, wohnhaft in der Ber!iner Str aße, <ZIP> <CITY>, wurd.e am <DATE> in die <LOCATION> in <NAME> öln aufgenom men. Er kLagte über a nhaltende Ma.genbeschwer den und wurde zur Untersuchung eingew1esen. Wäh rend seines Aufenthalts wurde e!ne endosk <NAME> Untersu chung (OPS: 8-980.2) durchgeführt, die keinen pathologischen Be fund erg ab. Zur Behand lun g seines Typ-2-Diabetes (Ell.9) wurde ihm Metformin erordn et. Seine Ehefr au, Frau <NAME> na <NAME> r, wohnhaft in der Rheinstraße <AGE>, ist a!s Notfallkontak t unter der Nummer +49 <MEDICALRECORD> errei chbar. Herr <NAME> wu rde nach erfolgrei cher Stabilisierung seines Gesund heitszustands am <DATE> entlas sen.","Herr Hansdieter Hauffer wagenknecht (1D: G1881162W6C), 69 Jahre alt, wohnhaft in der Ber!iner Str aße, 61330 Pößneck, wurd.e am Sep tember 1824 in die Lina-holzapfel-straße 12 in FALK öln aufgenom men. Er kLagte über a nhaltende Ma.genbeschwer den und wurde zur Untersuchung eingew1esen. Wäh rend seines Aufenthalts wurde e!ne endosk steffi Untersu chung (OPS: 8-980.2) durchgeführt, die keinen pathologischen Be fund erg ab. Zur Behand lun g seines Typ-2-Diabetes (Ell.9) wurde ihm Metformin erordn et. Seine Ehefr au, Frau Lina na Filiz r, wohnhaft in der Rheinstraße 15, ist a!s Notfallkontak t unter der Nummer +49 5588529 errei chbar. Herr Erol wu rde nach erfolgrei cher Stabilisierung seines Gesund heitszustands am 13.ç10.2024 entlas sen."


## Broken Text 2

In [ ]:
broken_text_2 = """Frau Helgé Sch mitz (Patienten-iD: T0110053F5D) wurde am 15. MaI 2024 im Städtischen Kran kenhaus in Düsseld orf aufgenommen. Sie kam mit Be schwerden über starke Rückenschmer zen und wurde zur Un tersuchung über wiesen. Die Dia gnose ergab Lu mbalgie (ICD-10: M54.5), und sie erhielt eine Ther a pie mit Ibup rofen (PZN: 12345678) zur Schmerzlinderung. Ihre S ozi alversicherungsnummer ist 123-45-6789. Während ihres Aufent halts wurde auch ein CT-S can durchge führt, um an dere mögliche Ursa chen auszuschließen. Frau Schmitz wurde am Maz 2024 entlassen, nachdem sich ihr Zustand de utlich verbessert hatte. Ihr E hemann, Tho mas Schmitz, wohnhaft in der Haupt straße 10, 40210 Düssel dorf."""

get_results(broken_text_2, deid_pipe)

,Sentence,Masked,Obfuscated
0,"Frau Helgé Sch mitz (Patienten-iD: T0110053F5D) wurde am 15. MaI 2024 im Städtischen Kran kenhaus in Düsseld orf aufgenommen. Sie kam mit Be schwerden über starke Rückenschmer zen und wurde zur Un tersuchung über wiesen. Die Dia gnose ergab Lu mbalgie (ICD-10: M54.5), und sie erhielt eine Ther a pie mit Ibup rofen (PZN: 12345678) zur Schmerzlinderung. Ihre S ozi alversicherungsnummer ist 123-45-6789. Während ihres Aufent halts wurde auch ein CT-S can durchge führt, um an dere mögliche Ursa chen auszuschließen. Frau Schmitz wurde am Maz 2024 entlassen, nachdem sich ihr Zustand de utlich verbessert hatte. Ihr E hemann, Tho mas Schmitz, wohnhaft in der Haupt straße 10, 40210 Düssel dorf.","Frau <NAME> <NAME> mitz (Patienten-iD: <ID>) wurde am 15. <DATE> im <HOSPITAL> in <CITY> orf aufgenommen. Sie kam mit Be schwerden über starke Rückenschmer zen und wurde zur Un tersuchung über wiesen. Die <NAME> gnose ergab <NAME> mbalgie (ICD-10: M54.5), und sie erhielt eine Ther a pie mit Ibup rofen (PZN: 12345678) zur Schmerzlinderung. Ihre S <NAME> alversicherungsnummer ist <PHONE>. Während ihres Aufent halts wurde auch ein CT-S can durchge führt, um an dere mögliche Ursa chen auszuschließen. Frau <NAME> wurde am <NAME> <DATE> entlassen, nachdem sich ihr Zustand <NAME> verbessert hatte. Ihr E hemann, <PATIENT>, wohnhaft in der Haupt straße 10, <ZIP> <LOCATION>.","Frau Carla Amelie mitz (Patienten-iD: G1881162W6C) wurde am 15. Juli 2024 im St. Benedikt Krankenhaus in Neustadtnerwaldnaab orf aufgenommen. Sie kam mit Be schwerden über starke Rückenschmer zen und wurde zur Un tersuchung über wiesen. Die Rotraut gnose ergab Irma mbalgie (ICD-10: M54.5), und sie erhielt eine Ther a pie mit Ibup rofen (PZN: 12345678) zur Schmerzlinderung. Ihre S emilie alversicherungsnummer ist 852-96-3074. Während ihres Aufent halts wurde auch ein CT-S can durchge führt, um an dere mögliche Ursa chen auszuschließen. Frau Hans-jürgen wurde am Edeltraut 2025 entlassen, nachdem sich ihr Zustand stumpf teichmann verbessert hatte. Ihr E hemann, Nikolaos henck Hans-jürgen, wohnhaft in der Haupt straße 10, 91581 Hans-otto-trub-platz 7."


## Broken Text 3

In [ ]:
broken_text_3 = """Um Ire Identität zu üerprüfen, benötigen wir Ire Kontonummer: DE44 5001 0517 5407 0000 00. Bitte geben Se auch Ihre Soziaversicherungsnmmer (SSN) im Rahmen des Antragsprozesses an: 289110478M657. Falls es dazu noch Fragen gibt, können Se sich gerne an Dr. Anna Schm*dt im Gesundheitszenrum Frankurt wenden. Diese befindet sich in der 60313 Frankf*urt am Main, Hessen, Deutschnd. Es besteht auch die Mglichkeit, das Büro unter +49-89-12345678 telefonisch zu erre*chen.

Neben der Frankurter Filiale gibt es eine wetere Zweigstelle in 12345 Hamb*rg, die Sie besuchen können. Die 28-jährige Frau, die wir früher in diesem Zammenhang erwähn haben, leidet unter einer Vogeschichte von Schwngerschaftsdiabtes, der vor acht Jahren dignostiziert wurde."""

get_results(broken_text_3, deid_pipe)

,Sentence,Masked,Obfuscated
0,"Um Ire Identität zu üerprüfen, benötigen wir Ire Kontonummer: DE44 5001 0517 5407 0000 00. Bitte geben Se auch Ihre Soziaversicherungsnmmer (SSN) im Rahmen des Antragsprozesses an: 289110478M657. Falls es dazu noch Fragen gibt, können Se sich gerne an Dr. Anna Schm*dt im Gesundheitszenrum Frankurt wenden. Diese befindet sich in der 60313 Frankf*urt am Main, Hessen, Deutschnd. Es besteht auch die Mglichkeit, das Büro unter +49-89-12345678 telefonisch zu erre*chen.\n\nNeben der Frankurter Filiale gibt es eine wetere Zweigstelle in 12345 Hamb*rg, die Sie besuchen können. Die 28-jährige Frau, die wir früher in diesem Zammenhang erwähn haben, leidet unter einer Vogeschichte von Schwngerschaftsdiabtes, der vor acht Jahren dignostiziert wurde.","Um Ire Identität zu üerprüfen, benötigen wir <LOCATION> Kontonummer: <ACCOUNT>. Bitte geben Se auch Ihre Soziaversicherungsnmmer (SSN) im Rahmen des Antragsprozesses an: <MEDICALRECORD>. Falls es dazu noch Fragen gibt, können Se sich gerne an Dr. <NAME> <NAME> im <LOCATION> wenden. Diese befindet sich in der <ZIP> <HOSPITAL>, <STATE>, <LOCATION>. Es besteht auch die Mglichkeit, das Büro unter <PHONE> telefonisch zu erre*chen.\n\nNeben der <LOCATION> gibt es eine wetere Zweigstelle in <ZIP> <PATIENT>, die Sie besuchen können. Die <AGE>-jährige Frau, die wir früher in diesem Zammenhang erwähn haben, leidet unter einer Vogeschichte von Schwngerschaftsdiabtes, der vor acht Jahren dignostiziert wurde.","Um Ire Identität zu üerprüfen, benötigen wir Schweitzerring 65 Kontonummer: CZ99 6118 1680 6910 1111 11. Bitte geben Se auch Ihre Soziaversicherungsnmmer (SSN) im Rahmen des Antragsprozesses an: 574881907B360. Falls es dazu noch Fragen gibt, können Se sich gerne an Dr. Ina Sylwia im Schlosserring 019 wenden. Diese befindet sich in der 31282 St. Kilian Klinik, Nordrhein-westfalen, Wendering 275. Es besteht auch die Mglichkeit, das Büro unter +94-74-85296307 telefonisch zu erre*chen.\n\nNeben der Patbergstraße 51 gibt es eine wetere Zweigstelle in 85296 Tlustek, die Sie besuchen können. Die 21-jährige Frau, die wir früher in diesem Zammenhang erwähn haben, leidet unter einer Vogeschichte von Schwngerschaftsdiabtes, der vor acht Jahren dignostiziert wurde."
